## Trade Simulator as a solution to Set Partitioning

In [9]:
import requests
import pandas as pd
import numpy as np
from dimod import BinaryQuadraticModel
from dwave.system import LeapHybridSampler
from random import randrange, sample
from dimod import ExactSolver

In [86]:
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])


slim_elements_df = elements_df[['second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]
df = slim_elements_df.merge(elements_types_df, left_on = 'element_type', right_on = 'id')

df = df[['plural_name_short','second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]

df = df.merge(teams_df, left_on = 'team', right_on = 'id')

df = df[['name', 'plural_name_short','second_name','team','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]

df = df[df.minutes > 100]
df.loc[df.plural_name_short == 'GKP', 'class'] = 1
df.loc[df.plural_name_short == 'DEF', 'class'] = 2
df.loc[df.plural_name_short == 'FWD', 'class'] = 3
df.loc[df.plural_name_short == 'MID', 'class'] = 4

df.head()

,name,plural_name_short,second_name,team,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,class
0,Arsenal,GKP,Leno,1,2.3,50,3131,0,26.2,131,1.0
2,Arsenal,MID,Borges Da Silva,1,0.3,65,1398,0,12.0,78,4.0
3,Arsenal,MID,Xhaka,1,0.5,50,2519,0,14.0,70,4.0
4,Arsenal,MID,El Sayed Elneny,1,0.7,45,1538,0,11.1,50,4.0
5,Arsenal,MID,Maitland-Niles,1,0.1,55,1714,0,12.4,68,4.0


In [80]:
df.name.unique()

array(['Arsenal', 'Aston Villa', 'Brighton', 'Burnley', 'Chelsea',
       'Crystal Palace', 'Everton', 'Leicester', 'Leeds', 'Liverpool',
       'Man City', 'Man Utd', 'Newcastle', 'Norwich', 'Southampton',
       'Spurs', 'Watford', 'West Ham', 'Wolves'], dtype=object)

In this notebook we run a trade simulator 100 times to see possible interations between our favourite team (Arsenal) and other teams in the Premier league. We use DWave's exact solver to find the solution for the set partitioning between (for example) 2 players on each team changing sides in some way (one player may change teams or both depending on the solution). We added a custom built in constraint that requires Arsenal to come out ahead at least 1% in any trade, and doesn't allow the opposing team to lose more than 10% on the deal. 

In [73]:

def get_trades(num_players, team, df):
    your_team = df[df.name == team]

    class_team = your_team['class'].iloc[0]
    df = df[df.team != class_team]
    tm_dict = dict(tuple(df.groupby('team')))
    
    for i in range(5000):

        trade_list1 = arsenal_players_df.iloc[sample(range(0, len(arsenal_players_df)), num_players)][['plural_name_short', 'value_season', 'second_name']].values.tolist()

        random_num = randrange(2,18)
        
        if random_num == 3:
            break
        else:
            try:
                trade_list2 =  tm_dict[random_num].iloc[sample(range(0, len(tm_dict[random_num])), num_players)][['plural_name_short', 'value_season', 'second_name']].values.tolist()
            except:
                break
        ####constraint on players allowed in trade###
        values1 = []
        values2 = []
        
        for i in trade_list1:
            values1.append(float(i[1]))
            
        for i in trade_list2:
            values2.append(float(i[1]))
        
        
        if (sum(values2) >= 1.01*sum(values1)) & (sum(values2) < 1.1*sum(values1)):

            values = values1 + values2
            
            names1 = []
            names2 = []
            
            for i in trade_list1:
                names1.append(i[2])
                
            for i in trade_list2:
                names2.append(i[2])
                
            pos1 = []
            pos2 = []
            
            for i in trade_list1:
                pos1.append(i[0])
                
            for i in trade_list2:
                pos2.append(i[0])
            
            names = names1 + names2
            positions = pos1 + pos2

            bqm = BinaryQuadraticModel('BINARY')
            n = len(values)
            x = {i: bqm.add_variable(f'x_{i}') for i in range(n)}

            bqm.add_linear_equality_constraint(
                [(x[i], 2.0 * values[i]) for i in range(n)],
                constant=-sum(values),
                lagrange_multiplier=10
            )

            response = ExactSolver().sample(bqm).truncate(5)
            solution = response.first.sample

            set1 = {values[i] for i in x if solution[x[i]]}
            set2 = {values[i] for i in x if not solution[x[i]]}

            set1_names = {(names[i], positions[i]) for i in x if solution[x[i]]}
            set2_names = {(names[i], positions[i]) for i in x if not solution[x[i]]}

            print('Breaking News! A Trade occured!')
            print(' ')
            print('Before')
            print('Players on Arsenal: ' + str(trade_list1))
            print('Players on Team ' + str(tm_dict[random_num]['name'].iloc[0]) + ' ' + str(trade_list2))
            print(' ')
            print('After')


            print(f'Sum of value: {sum(set1)}; players now on Arsenal {tuple(set1_names)}')
            print(f'Sum of value: {sum(set2)}; players on ' + str(tm_dict[random_num]['name'].iloc[0]) + ' '  +  f'{tuple(set2_names)}')
            print(' ')

        else:
            pass

***may have to run more than once due to constraints***

In [75]:
get_trades(1, 'Aston Villa', df)

Breaking News! A Trade occured!
 
Before
Players on Arsenal: [['DEF', '19.2', 'Tierney']]
Players on Team Spurs [['FWD', '19.4', 'Kane']]
 
After
Sum of value: 19.2; players now on Arsenal (('Tierney', 'DEF'),)
Sum of value: 19.4; players on Spurs (('Kane', 'FWD'),)
 
Breaking News! A Trade occured!
 
Before
Players on Arsenal: [['MID', '11.1', 'El Sayed Elneny']]
Players on Team Crystal Palace [['MID', '11.5', 'Milivojevic']]
 
After
Sum of value: 11.1; players now on Arsenal (('El Sayed Elneny', 'MID'),)
Sum of value: 11.5; players on Crystal Palace (('Milivojevic', 'MID'),)
 


In [82]:
get_trades(2, 'Arsenal', df)

Breaking News! A Trade occured!
 
Before
Players on Arsenal: [['MID', '15.2', 'Pépé'], ['MID', '9.4', 'Partey']]
Players on Team Man Utd [['MID', '13.8', 'Rodrigues de Paula Santos'], ['FWD', '11.6', 'Cavani']]
 
After
Sum of value: 24.6; players now on Arsenal (('Pépé', 'MID'), ('Partey', 'MID'))
Sum of value: 25.4; players on Man Utd (('Rodrigues de Paula Santos', 'MID'), ('Cavani', 'FWD'))
 
Breaking News! A Trade occured!
 
Before
Players on Arsenal: [['FWD', '13.1', 'Aubameyang'], ['MID', '14.0', 'Xhaka']]
Players on Team Aston Villa [['DEF', '25.6', 'Mings'], ['MID', '2.0', 'Sanson']]
 
After
Sum of value: 27.1; players now on Arsenal (('Xhaka', 'MID'), ('Aubameyang', 'FWD'))
Sum of value: 27.6; players on Aston Villa (('Sanson', 'MID'), ('Mings', 'DEF'))
 


In [85]:
get_trades(3, 'Liverpool', df)

Breaking News! A Trade occured!
 
Before
Players on Arsenal: [['DEF', '18.8', 'Bellerín'], ['MID', '13.2', 'Willock'], ['MID', '15.2', 'Pépé']]
Players on Team Brighton [['MID', '19.3', 'Groß'], ['GKP', '22.4', 'Sánchez'], ['GKP', '8.2', 'Ryan']]
 
After
Sum of value: 49.4; players now on Arsenal (('Bellerín', 'DEF'), ('Sánchez', 'GKP'), ('Ryan', 'GKP'))
Sum of value: 47.7; players on Brighton (('Pépé', 'MID'), ('Groß', 'MID'), ('Willock', 'MID'))
 
